In [1]:
!apt-get -y install openjdk-8-jdk-headless
!wget http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar xf spark-2.4.0-bin-hadoop2.7.tgz
!pip install pyspark_dist_explore findspark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
openjdk-8-jdk-headless is already the newest version (8u191-b12-2ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
--2019-03-06 02:08:28--  http://apache.osuosl.org/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
Resolving apache.osuosl.org (apache.osuosl.org)... 140.211.166.134, 64.50.233.100, 64.50.236.52, ...
Connecting to apache.osuosl.org (apache.osuosl.org)|140.211.166.134|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 227893062 (217M) [application/x-gzip]
Saving to: ‘spark-2.4.0-bin-hadoop2.7.tgz’

spark-2.4.0-bin-had 100%[===================>] 217.33M  71.3MB/s    in 3.0s    

2019-03-06 02:08:31 (71.3 MB/s) - ‘spark-2.4.0-bin-hadoop2.7.tgz’ saved [227893062/227893062]



In [0]:
import os

# setup environment variables
os.environ["JAVA_HOME"] = '/usr/lib/jvm/java-8-openjdk-amd64'
os.environ["SPARK_HOME"] = '/content/spark-2.4.0-bin-hadoop2.7'
os.environ['KAGGLE_USERNAME'] = 'ronaldsumbayak'
os.environ['KAGGLE_KEY'] = '771009dffffa0cd0883d5fb98594d756'

In [0]:
# download dataset
!kaggle datasets download -d kmader/aminer-academic-citation-dataset
!unzip aminer-academic-citation-dataset.zip -d data

100% 2.27G/2.27G [00:41<00:00, 15.5MB/s]
100% 2.27G/2.27G [00:41<00:00, 58.4MB/s]
Archive:  aminer-academic-citation-dataset.zip
  inflating: data/dblp-ref-1.json    
  inflating: data/citation-network2.txt  

In [0]:
import findspark
findspark.init()

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark_dist_explore import hist
import matplotlib.pyplot as plt

In [0]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [0]:
from functools import reduce

dblp_refs = [spark.read.json('data/dblp-ref-0.json'),
             spark.read.json('data/dblp-ref-1.json'),
             spark.read.json('data/dblp-ref-2.json'),
             spark.read.json('data/dblp-ref-3.json')]
df = reduce(lambda x, f: f.union(x), dblp_refs)

In [0]:
df.count()

In [0]:
df.show()

In [0]:
annual_paper_count = df.groupBy('year').count().orderBy('year', ascending=False)

In [0]:
annual_paper_count.show()

In [0]:
annual_paper_count.toPandas().plot(x='year', y='count')

In [0]:
n_citation_avg = df.agg(avg('n_citation')).collect()[0][0]
print('Average number of citation per paper:', int(n_citation_avg))

In [0]:
yv = df.groupBy('year', 'venue') \
       .agg(count('*').alias('count')) \
       .where('venue != ""')
ym = df.groupBy('year', 'venue') \
       .agg(count('*').alias('count')) \
       .where('venue != ""') \
       .groupBy('year') \
       .agg(max('count').alias('count'))

vy = ym.join(yv, ['year', 'count'], 'left') \
       .orderBy('year', ascending=False)
vy.show()